In [69]:
import numpy as np
import pandas as pd


In [70]:
"""
  Extend the functionality of a Decision Tree Classifier with numpy integration:
  Basic Idea comes from: https://github.com/random-forests/tutorials/blob/master/decision_tree.ipynb
  Numpy and Pandas Extrension from: https://github.com/Nir-J/Decision_tree_ID3/blob/master/dtree.ipynb
  Added Depth, min_sample_leafs and min_sample_Split from: https://medium.datadriveninvestor.com/easy-implementation-of-decision-tree-with-python-numpy-9ec64f05f8ae
  
  Decision Tree Regression Idea from: https://austindavidbrown.github.io/post/2019/01/regression-decision-trees-in-python/
"""

'\n  Extend the functionality of a Decision Tree Classifier with numpy integration:\n  Basic Idea comes from: https://github.com/random-forests/tutorials/blob/master/decision_tree.ipynb\n  Numpy and Pandas Extrension from: https://github.com/Nir-J/Decision_tree_ID3/blob/master/dtree.ipynb\n  Added Depth, min_sample_leafs and min_sample_Split from: https://medium.datadriveninvestor.com/easy-implementation-of-decision-tree-with-python-numpy-9ec64f05f8ae\n  \n  Decision Tree Regression Idea from: https://austindavidbrown.github.io/post/2019/01/regression-decision-trees-in-python/\n'

In [71]:
"""
  Decision Tree:
  @Input: 
    - DataFrame with size: (rows,cols)
    - the last column is the target label 
    - Label is either numeric (class number) or string type ("Apple")
    - features are of either numeric or string type
"""

'\n  Decision Tree:\n  @Input: \n    - DataFrame with size: (rows,cols)\n    - the last column is the target label \n    - Label is either numeric (class number) or string type ("Apple")\n    - features are of either numeric or string type\n'

In [72]:
class Utils:
  def __init__(self):
    pass

  def unique_vals(df,col):
    """ find unique values for a column in a dataset """
    return np.unique(df.loc[:,col])
  def class_counts(df):
    """ counts the number of each class of target label in a dataset """
    return np.unique(df.iloc[:,-1],return_counts=True)

  def is_numeric(val):
    """ test if a value is numeric """
    return isinstance(val,np.int64) or isinstance(val,np.float) or isinstance(val,np.int) or isinstance(val,int)

In [73]:
class Question:
  def __init__(self,col_num,val):
    self.col_num = col_num
    self.eval_val = val
  
  def match(self,df_row):
    """
      @input: 1-d DataFrame row
    """
    test_val = df_row[self.col_num] 
    if Utils.is_numeric(test_val):
      # we choose ">=" as split criterion for numeric values
      return test_val >= self.eval_val
    else:
      # we choose "==" as split criterion for string values
      return test_val == self.eval_val

  def partition(self,rows):
    """partition the dataset with given question
    """
    true_rows,false_rows = [],[]
    for row in rows:
      if self.match(df_row):
        true_rows.append(row)
      else:
        false_rows.append(row)
    return true_rows,false_rows

  def df_partition(self,rows):
    """@input: pd.DataFrame
    @return: pd.DataFrame
    """
    if Utils.is_numeric(self.eval_val):
      true_rows,false_rows = rows[rows.iloc[:,self.col_num]>=self.eval_val],rows[rows.iloc[:,self.col_num]<self.eval_val]
    else:
      #print(rows[self.col_num]==self.eval_val)
      true_rows,false_rows = rows[rows.iloc[:,self.col_num]==self.eval_val],rows[~(rows.iloc[:,self.col_num]==self.eval_val)]
    
    return true_rows,false_rows

  def classify(self,row):
    """@Input:pandas.Series"""
    if Utils.is_numeric(self.eval_val):
      return row.iloc[self.col_num]>=self.eval_val
    else:
      #print(rows[self.col_num]==self.eval_val)
      return row.iloc[self.col_num]==self.eval_val


  def __repr__(self):
    criterion = "=="
    if Utils.is_numeric(self.eval_val):
      criterion = ">="
    return f"Is {df.columns[self.col_num]} {criterion} {self.eval_val} "

In [74]:
class Gain:
  def gini(rows):
    """@input: pd.DataFrame
    """
    names,counts = Utils.class_counts(rows)
    impurity = 1.0
    prob_lbl = np.sum((counts/float(len(rows)))**2)
    impurity -= prob_lbl

    return impurity

  def entropy(rows):
    """
      entropy calculation from label of current rows
    """
    names,counts = Utils.class_counts(rows)
    entropy = 0
    prob_of_lbl = np.sum(-counts/float(len(rows))*np.log2(counts/float(len(rows))))
    entropy -= prob_of_lbl
    return entropy

  def info_gain_gini(left,right,current_uncertainty):
    """
      @input:
        left: true rows
        right: false rows
        current_uncertainty: gini of current rows
    """
    p = float(len(left)) / (len(left)+len(right))
    return current_uncertainty - p*Gain.gini(left) -(1-p)*Gain.gini(right)

  def info_gain_entropy(left,right,current_uncertainty):
    """
      @input:
        left: true rows
        right: false rows
        current_uncertainty: entropy of current rows
    """
    p = float(len(left))/(len(left)+len(right)) # percent of left
    return current_uncertainty - p*Gain.entropy(left) -(1-p)*Gain.entropy(right)

In [75]:
class Leaf:
    """A Leaf node classifies data.

    This holds a dictionary of class (e.g., "Apple") -> number of times
    it appears in the rows from the training data that reach this leaf.
    """

    def __init__(self, rows): 
      # counts = np.column_stack(Utils.class_counts(rows)) # == list(zip(v1,v2))
      self.prediction = np.mean(rows.iloc[:,-1]) # mean value as regression value
class Node:
    """A Decision Node tests a condition.

    This holds a reference to the condition, and to the two child nodes.
    """

    def __init__(self,condition,
              true_branch,
              false_branch):
        self.condition = condition
        self.true_branch = true_branch
        self.false_branch = false_branch

In [76]:
class DecisionTreeRegressorMy:
  def __init__(self,max_depth=32,min_sample_split=2):
    self.max_depth = max_depth
    self.min_sample_split = min_sample_split

  def train(self,rows):
    """ build a tree with given data"""
    self.root = self._build_tree(rows,0)
    return self.root

  def find_best_split(self,rows):
    """
    Find the best question to ask by iterating over every feature / value
    and calculating the information gain.
    
    @input:
      rows: pd.DataFrame
    """
    best_gain = 0  # keep track of the best information gain
    best_question = None  # keep train of the feature / value that produced it
    current_uncertainty = Gain.gini(rows)
    n_features = len(rows.columns)-1  # number of features

    for col in range(n_features):  # for each feature

        values = np.unique(rows.iloc[:,col]) # unique values in the column
        for val in values:  
            question = Question(col, val)
            # try splitting the dataset
            true_rows, false_rows = question.df_partition(rows)

            # Skip this split if it doesn't divide the dataset.
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            # Calculate the information gain from this split
            gain = Gain.info_gain_gini(true_rows, false_rows, current_uncertainty)

            if gain >= best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question

  def _build_tree(self,rows,depth):
    """
      Recurvisely split trees and test condition

      @input: pd.DataFrame
    """
    # if number of samples is less than required, we cannot split
    if df.shape[0] < self.min_sample_split:
      return Leaf(rows)

    gain, question = self.find_best_split(rows)

    # at the recursion end: no gain obtained
    # or depth violated
    # --> return leaf
    if gain == 0 or depth == self.max_depth:
        return Leaf(rows)

    # partition recursively with best condition and pass "depth" control
    true_rows, false_rows = question.df_partition(rows)
    true_branch = self._build_tree(true_rows,depth+1) # node true
    false_branch = self._build_tree(false_rows,depth+1) # node false

    print("splitting currend node depth %d"%(depth))

    return Node(question, true_branch, false_branch)



  def predict_one(self,row):
    """ return prediction for single row """
    prediction = self._classify(row,self.root)
    return prediction

  def predict(self,rows):
    predictions = []
    for idx in rows.index:
      print(idx)
      predictions.append(self.predict_one(rows.loc[idx,:]))
    return predictions


  def _classify(self,row,node):
    # base case: at a leaf node
    if isinstance(node,Leaf):
      
      return node.prediction

    # climb down the tree with condition
    if isinstance(node,Node):
      condition = node.condition.classify(row)
      
      if condition:
        return self._classify(row,node.true_branch) # need add return at each branch if we want a return at the end
      else:
        return self._classify(row,node.false_branch)
    

  def print_tree(self,node, spacing=""):
    """World's most elegant tree printing function."""

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        print (spacing + "Predict", node.predictions)
        return

    # Print the question at this node
    print (spacing + str(node.condition))

    # Call this function recursively on the true branch
    print (spacing + '--> True:')
    self.print_tree(node.true_branch, spacing + "  ")

    # Call this function recursively on the false branch
    print (spacing + '--> False:')
    self.print_tree(node.false_branch, spacing + "  ")

In [77]:
class Metrics:
  def accuracy(preds,labels):
    """Calculate acc given two numpy arrays in percent"""
    return np.mean(preds == labels)*100

  def confusion_matrix(preds,labels):
    """Calculate the confusion matrix
      @properties:
        Maximal 10 classes
        if class=2: TP,FP matrix
    """
    matrix = np.zeros((10,10))
    for i in range(len(preds)):
      matrix[preds[i],labels[i]]+=1
    return matrix

  def precision_and_recall(preds,labels):
    """Returns individual Precion and Recall values of each class
    
      @Properties:
        Precision: TP/TP+FP
        Recall: TP/TP+FN
        f1: 2*P*R/P+R

      @Confusion Matrix:class=2
      TP|FP
      FN|FP
        
    """
    matrix = Metrics.confusion_matrix(preds,labels)
    r,p =[],[]
    for i in range(10):
      TP = float(matrix[i,i])
      TP_FP = np.sum(matrix[i,:])
      TP_FN = np.sum(matrix[:,i])

      recall = np.nan if TP_FN == 0 else TP/TP_FN
      precision = np.nan if TP_FP == 0 else TP/TP_FP
      r.append(recall)
      p.append(precision)
    
    return (p, r)

  def precision_and_recall_score(preds,labels):
    """Calculate Macro Version of Precision and Recall"""
    p,r = Metrics.precision_and_recall(preds,labels)
    p_score,r_score = 0,0
    last_class_index = 0
    for i,val in enumerate(p):
      if ~np.isnan(val): 
        p_score += p[i]
        if i>last_class_index:
          last_class_index = i
    for i,val in enumerate(r):
      if ~np.isnan(val):
        r_score += r[i]
        if i>last_class_index:
          last_class_index = i
    class_count = last_class_index +1  
        
    return float(p_score/class_count),float(r_score/class_count)

  def f1score(preds, labels):
    """Calculates macro f1 score given two numpy arrays"""
    
    precision,recall = Metrics.precision_and_recall_score(preds,labels)
    return 2*precision*recall/(precision+recall)

In [78]:
# test_metrics
from sklearn.metrics import precision_score,recall_score,f1_score
a = [0,1,0,1,1,2]
b = [0,1,1,1,1,3]
print(Metrics.confusion_matrix(a,b))
print(Metrics.precision_and_recall(a,b))
print(Metrics.precision_and_recall_score(a,b))
print(Metrics.f1score(a,b))
print(precision_score(a,b,average='macro'))
print(recall_score(a,b,average='macro'))
print(f1_score(a,b,average='macro'))


[[1. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 3. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
([0.5, 1.0, 0.0, nan, nan, nan, nan, nan, nan, nan], [1.0, 0.75, nan, 0.0, nan, nan, nan, nan, nan, nan])
(0.375, 0.4375)
0.40384615384615385
0.4375
0.375
0.38095238095238093


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# test main

In [79]:
# test_main

_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 3, 'Apple'],
    ['Red', 1, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],
] 

df = pd.DataFrame(_data,columns=["color", "diameter", "label"])
print(df.shape)
test_df = pd.DataFrame([['Green', 3, 'Apple']],columns=["color", "diameter", "label"])


print(df.loc[:,"color"])
print(["red","green"]!="green")

q1 = Question(1,2)
"""
true,false =q1.df_partition(df)
print(true)
print(false)

no_mixing = [['Apple'],['Orange']]
no = pd.DataFrame(no_mixing)
gg = Gain()
print(gg.gini(no))

print(Gain.gini(df))

dt = DecisionTreeClassifier()
mytree = dt.train(df)
print(mytree)
dt.print_tree(mytree)

print(dt.predict(test_df))
"""

(5, 3)
0     Green
1    Yellow
2       Red
3       Red
4    Yellow
Name: color, dtype: object
True


"\ntrue,false =q1.df_partition(df)\nprint(true)\nprint(false)\n\nno_mixing = [['Apple'],['Orange']]\nno = pd.DataFrame(no_mixing)\ngg = Gain()\nprint(gg.gini(no))\n\nprint(Gain.gini(df))\n\ndt = DecisionTreeClassifier()\nmytree = dt.train(df)\nprint(mytree)\ndt.print_tree(mytree)\n\nprint(dt.predict(test_df))\n"

In [80]:
# generate random samples
def generate_random_regression(n_samples=100,n_features=10):
  """ implement: y=kx+b+noise """
  intercept = 5*np.ones(n_features)
  B = 3*np.ones((n_features,1))
  X = np.zeros((n_samples,n_features))
  for i in range(0,n_samples):
    X[i,:] = np.random.multivariate_normal(np.zeros(n_features),10*np.identity(n_features)) #mean=0,cov=identity matrix
  noise = np.random.multivariate_normal(np.zeros(n_features),np.identity(n_features)) 
  print(X.shape,B.shape)
  y = intercept + X@B + noise # @ = matmul

  #train test split
  n_train = int(.7*n_samples)
  I = np.arange(0,n_samples)
  train_idx = np.random.choice(I,n_train,replace=False)
  test_idx = np.setdiff1d(I,train_idx)

  return X[train_idx,:],y[train_idx],X[test_idx,:],y[test_idx]

def generate_random_classifier(n_samples=100,n_features=10):
  """ implement: y=kx+b+noise """
  intercept = 5*np.ones(n_features)
  B = 3*np.ones((n_features,1))
  X = np.zeros((n_samples,n_features))
  for i in range(0,n_samples):
    X[i,:] = np.random.multivariate_normal(np.zeros(n_features),10*np.identity(n_features)) #mean=0,cov=identity matrix
  y = np.random.randint(low=0,high=2,size=(n_samples,1)) # @ = matmul/dot in 2-D

  #train test split
  n_train = int(.7*n_samples)
  I = np.arange(0,n_samples)
  train_idx = np.random.choice(I,n_train,replace=False)
  test_idx = np.setdiff1d(I,train_idx)

  return X[train_idx,:],y[train_idx],X[test_idx,:],y[test_idx]

xtrain,ytrain,xtest,ytest = generate_random_classifier()


In [81]:
df = pd.read_csv("sample_data/california_housing_train.csv")
print(df.head())

n_samples = df.shape[0]
n_train = int(.7*n_samples)
I = np.arange(0,n_samples)
train_idx = np.random.choice(I,n_train,replace=False)
test_idx = np.setdiff1d(I,train_idx)

xtrain,ytrain,xtest,ytest = df.iloc[train_idx,:-1],df.iloc[train_idx,-1],df.iloc[test_idx,:-1],df.iloc[test_idx,-1]

   longitude  latitude  ...  median_income  median_house_value
0    -114.31     34.19  ...         1.4936             66900.0
1    -114.47     34.40  ...         1.8200             80100.0
2    -114.56     33.69  ...         1.6509             85700.0
3    -114.57     33.64  ...         3.1917             73400.0
4    -114.57     33.57  ...         1.9250             65500.0

[5 rows x 9 columns]


In [82]:
np.mean(xtest.iloc[:,-1])

3.9010790392156847

In [83]:
dt = DecisionTreeRegressorMy(max_depth=10)
mytree = dt.train(df.iloc[train_idx])
# dt.print_tree(mytree)

splitting currend node depth 6
splitting currend node depth 5
splitting currend node depth 5
splitting currend node depth 4
splitting currend node depth 9
splitting currend node depth 8
splitting currend node depth 9
splitting currend node depth 8
splitting currend node depth 7
splitting currend node depth 7
splitting currend node depth 6
splitting currend node depth 8
splitting currend node depth 9
splitting currend node depth 9
splitting currend node depth 8
splitting currend node depth 7
splitting currend node depth 9
splitting currend node depth 8
splitting currend node depth 9
splitting currend node depth 8
splitting currend node depth 7
splitting currend node depth 6
splitting currend node depth 5
splitting currend node depth 7
splitting currend node depth 6
splitting currend node depth 7
splitting currend node depth 6
splitting currend node depth 5
splitting currend node depth 4
splitting currend node depth 3
splitting currend node depth 9
splitting currend node depth 8
splittin

# myRegressor

In [ ]:
row = xtrain.iloc[0]
print(row)
row.iloc[0]

In [84]:
y_pred = dt.predict(xtrain)

流式输出内容被截断，只能显示最后 5000 行内容。
6593
13963
1775
5521
5739
12673
15257
914
15216
11046
2686
3861
4913
2243
16788
15082
1987
2219
9623
9338
644
14716
11148
8223
16538
11643
14247
9875
6938
10091
15721
8169
8817
2899
7483
2840
7422
15556
9506
9683
16412
15151
2318
12710
11593
7981
6605
7975
136
5575
15657
11644
16254
9886
9678
12778
15735
4928
5585
12877
1262
15084
2036
16813
2582
2676
13336
4044
8315
2449
5485
15479
7668
5148
15919
6673
7367
15857
3672
13732
14124
15054
3059
1898
9194
6525
5968
10804
5306
10175
4381
4413
399
6904
14379
9216
4965
14673
3057
1686
966
16011
4679
7735
9614
3009
14380
7509
6149
1738
8288
5032
634
14975
9980
13783
12633
12328
7010
15641
2007
662
7230
6262
1570
4732
8468
16959
7016
7050
8615
16062
10063
11554
14818
15780
10377
5906
6802
13329
3653
11023
956
14409
8936
8577
16921
13950
142
11823
11739
12592
4728
3685
567
9778
9329
8644
16600
5989
12583
10053
11174
5312
3594
16997
11929
5374
1608
10433
15818
11655
8953
13788
1539
1834
8783
7468
270
10488
2240
1829
742

In [ ]:
y_pred

In [88]:
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor
from sklearn.metrics import mean_squared_error,mean_squared_log_error


dt = DecisionTreeRegressor(max_depth=10)
dt.fit(xtrain,ytrain)


DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=10,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

In [89]:
y_pred_sklearn = dt.predict(xtrain)
skacc = mean_squared_log_error(y_pred_sklearn,ytrain)
myacc = mean_squared_log_error(y_pred,ytrain)

print(skacc,myacc)

0.05352269311927534 0.12516167082149313
